In [4]:
import sys
import time
from Heuristics.secret import rpc_user, rpc_password
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()

In [5]:
#DB Query
import sqlite3

db_index = '/home/dnlab/BitcoinBlockSampler/index.db'
db_txhash = '/media/dnlab/0602da39-763c-42b0-b186-f929ac6b3f66/200529/txhash.db'
index_conn = sqlite3.connect(db_index)
txhash_conn = sqlite3.connect(db_txhash)
icur = index_conn.cursor()
tcur = txhash_conn.cursor()


def get_txid(txhash):
    try:
        icur.execute('''SELECT DISTINCT id FROM TxID WHERE txhash = '{}'; '''.format(txhash))
        tx_indexes = icur.fetchall()
        return tx_indexes[0][0]

    except Exception as e:
        return None


def get_addr_txin(tx_indexes):
    try:
        tcur.execute('''SELECT DISTINCT addr FROM TxIn WHERE tx = '{}'; '''.format(tx_indexes))
        address_list = [str(addr[0]) for addr in tcur.fetchall()]
        return set(address_list)

    except Exception as e:
        return None


def get_addr_txout(tx_indexes):
    try:
        tcur.execute('''SELECT DISTINCT addr FROM TxOut WHERE tx = '{}'; '''.format(tx_indexes))
        address_list = [str(addr[0]) for addr in tcur.fetchall()]
        return set(address_list)
    except Exception as e:
        return None



In [6]:
db_path = '/home/dnlab/BitcoinBlockSampler/cluster.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()


def create_meta_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Meta (
                     key TEXT PRIMARY KEY,
                     value INTEGER);
                ''')
    
    
def update_meta_table(key, value):
    cur.execute('''INSERT OR IGNORE INTO Meta (
                        key, value) VALUES (
                        ?, ?);
                ''', (key, value))
    cur.execute('''UPDATE Meta SET value = ? WHERE key = ?;
                ''', (value, key))
    

def get_meta(key):
    cur.execute('''SELECT value FROM Meta WHERE key = ?''', (key,))
    result = cur.fetchone()
    if result is not None:
        result = result[0]
    return result
    
    
def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     address INTEGER PRIMARY KEY,
                     number INTEGER NOT NULL);
                ''')
    
    
def insert_cluster(address, number):
    cur.execute('''INSERT OR IGNORE INTO Cluster (
                       address, number) VALUES (
                       ?, ?);
                    ''', (address, number))

    
def insert_cluster_many(addr_list):
    #print(addr_list)
    cur.executemany('''INSERT OR IGNORE INTO Cluster VALUES(?, ?)''',addr_list)
    
    
def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

    
def commit_transactions():
    cur.execute('COMMIT;')

    
def get_min_all_cluster(addrss):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ('{",".join(addrss)}')'''.replace('\'',''))
    return cur.fetchone()[0]


def get_min_clustered(addrss):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ('{",".join(addrss)}') and number > -1'''.replace('\'',''))
    return cur.fetchone()[0]


def get_cluster_number(addrss):
    cur.execute(f'''SELECT number FROM Cluster WHERE address IN ('{",".join(addrss)}')'''.replace('\'',''))
    cls_num = []
    for addr_tuple in cur.fetchall():
        cls_num.append(addr_tuple[0])
    return set(cls_num)


def get_all_cluster():
    try:
        cur.execute('''SELECT DISTINCT * FROM Cluster; ''')
        addr_dict = dict()
        for addr in cur.fetchall():
            addr_dict.update({addr[0]:addr[1]})
        return addr_dict
    except Exception as e:
        return None
create_meta_table()
create_cluster_table()

In [7]:
def is_mi_cond(in_addrs, out_addrs):
    if len(in_addrs) < 2:
        return False
    if len(out_addrs) > 2:
        return False
    return True

def all_same_cls_num(addrs):
    cls_num_set = set(get_cluster_number(addrs))
    if len(cls_num_set) == 1:
        return True
    else:
        return False
        
def get_min_cluster_num(addr, flag=0):
    ''' DON'T USE
        flag: 0 전체 최소값
        flag: 1 -1이 아닌 최소값'''
    cluster_num_list = get_min_cluster(addr)
    cluster_num_list = list()
    for addr in addr_set.keys():
        cluster_num_list.append(addr_set[addr])
    sort_cls_num_list = sorted(cluster_num_list)
    if flag == 0:
        return sort_cls_num_list[0]
    elif flag == 1:
        for num in sort_cls_num_list:
            if num > -1:
                return num
              
def get_cluster_num(addrs):
    cls_num = -1
    max_cluster_num  = get_meta('max_num')
    if all_same_cls_num(addrs):
        cls_num = get_min_all_cluster(addrs)
        if cls_num == -1:
            cls_num = max_cluster_num + 1
            max_cluster_num = cls_num
            #######start commit##########
            begin_transactions()
            update_meta_table('max_num', max_cluster_num)
            commit_transactions()
            #######start commit#########
    else:
        cls_num = get_min_clustered(addrs)
    return cls_num

def update_cluster(addrs, cluster_num):
    try:
        cluster_nums = [cluster_num] * len(addrs)
        cluster_list = list(zip(addrs, cluster_nums))
        #print(cluster_list)
        ####begintransaction######                 
        begin_transactions()
        insert_cluster_many(cluster_list)
        commit_transactions()
        ####end commit ###########
        return True
    except Exception as e:
        print(e)
        return False

In [8]:
def multi_input(height):
    
        if is_mi_cond(in_addrs, out_addrs):
            cluster_num = get_cluster_num(in_addrs)
            return in_addrs, cluster_num
        return None

In [9]:
def find_all_addr(height):
    try:
        block_hash = rpc_connection.getblockhash(height)
        txes = rpc_connection.getblock(block_hash)['tx']
    except Exception as e:
        print(e)
        return None
    
    for tx in txes:
        tx_indexes = get_txid(tx)
        in_addrs = get_addr_txin(tx_indexes)
        out_addrs = get_addr_txout(tx_indexes)
        
        all_addrs = list(in_addrs.union(out_addrs))
        cluster_nums = [-1] * len(all_addrs)
        cluster_list = list(zip(all_addrs, cluster_nums))
    return cluster_list

In [27]:
import multiprocessing
import sys
rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']
pool_num = multiprocessing.cpu_count()//2

term = 1000
start_height = 0
end_height = 480000

starttime = time.time()
for sheight, eheight in zip(range(start_height, end_height,term), range(start_height + term, end_height + term, term)):
    stime = time.time()
    if eheight >= end_height:
        eheight = end_height + 1

    with multiprocessing.Pool(pool_num) as p:
        ####begintransaction######                 
        begin_transactions()
        result = p.imap(find_all_addr, range(sheight, eheight))
        for tuple_element in result:
            print(tuple_element)
            if tuple_element != None:
                if len(tuple_element) != 0 :
                    insert_cluster_many(tuple_element)
        commit_transactions()
        ####end commit ###########   
    etime = time.time()
    print(f'TxIn Job done from {sheight} to {eheight-1} during {etime-stime}')

-1: JSON value is not a string as expected
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
string indices must be integers
string indices must be integers
-1: JSON value is not a string as expected
-1: JSON value is not a string as expected
string indices must be integers
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
string indices must be integers
-1: JSON value is not a string as expected
-1: JSON value is not a string as expected
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
string indices must be integers
-1: JSON value is not a string as expected
string indices must be integers
stri

Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent
Request-sent

TypeError: 'NoneType' object is not iterable

In [ ]:
import multiprocessing
import sys
rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
longest_height = rpc_connection.getblock(best_block_hash)['height']
pool_num = multiprocessing.cpu_count()//2

####begintransaction######
begin_transactions()
update_meta_table('max_num', -1)
commit_transactions()
####end commit ###########

term = 1000
start_height = 0
end_height = 480000

starttime = time.time()
for sheight, eheight in zip(range(start_height, end_height,term), range(start_height + term, end_height + term, term)):
    stime = time.time()
    if eheight >= end_height:
        eheight = end_height + 1

    with multiprocessing.Pool(pool_num) as p:
        result = p.imap(multi_input, range(sheight, eheight))
        for tuple_element in result:
            print(tuple_element)
            if tuple_element == None:
                continue
            in_addrs = tuple_element[0]
            cluster_num = tuple_element[1]
            update_cluster(in_addrs, cluster_num)
    etime = time.time()
    print(f'TxIn Job done from {sheight} to {eheight-1} during {etime-stime}')

In [26]:
commit_transactions()